# DATA PROCESSING

In [1]:
#Computation
import warnings
import numpy as np
import pandas as pd
import sklearn as skl
import scipy as sp
import re
import math
import os
import time
from datetime import datetime
import gc

from sklearn import metrics
from sklearn.mixture import GaussianMixture
from collections import defaultdict
from tqdm import tqdm

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.tools import add_constant
from statsmodels.stats.multitest import multipletests
from Bio import SeqIO

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
from adjustText import adjust_text

#Style
sns.set(style="white")
sns.set_palette('Paired')
%matplotlib inline

# Data file paths

file_dat_res from https://github.com/jurgjn/af2genomics/blob/main/notebooks/interaction_interfaces/somatic_mutations_protein_abundance_ifresid.tsv \
(accession 24.05.2024)

filte_dat_vep from `/cluster/work/beltrao/jjaenes/23.11.09_cptac_missense_mutations/cptac_missense_mutations_merge_23.11.1.tsv` \
(accession 10.11.2023)

file_biogrid_all from [BioGRID](https://downloads.thebiogrid.org/BioGRID)\
(accession 25.05.2023)\
Version 4.4.227\
File: BIOGRID-ALL-4.4.227.tab3.zip

file_biogrid_mv from [BioGRID](https://downloads.thebiogrid.org/BioGRID)\
(accession 26.05.2023)\
Version 4.4.227\
File: BIOGRID-MV-Physical-4.4.227.tab3.zip

file_string_all from [STRING](https://string-db.org/cgi/download?sessionId=bJrNsFgGvKiS&species_text=Homo+sapiens)\
(accession 01.11.2023)\
Version 12.0\
File: 9606.protein.links.v12.0.txt.gz

file_string_physical from [STRING](https://string-db.org/cgi/download?sessionId=bJrNsFgGvKiS&species_text=Homo+sapiens)\
(accession 01.11.2023)\
Version 12.0\
File: 9606.protein.physical.links.detailed.v12.0.txt.gz

file_translation_hgnc_update from [HGNC BioMart](https://biomart.genenames.org/martform/#!/default/HGNC?datasets=hgnc_gene_mart) (accession 31.10.2023) \
Locus group: protein-coding gene \
_Approved symbol, Alias symbol, Previous symbol_

file_translation_hgnc_uniprot from [HGNC BioMart](https://biomart.genenames.org/martform/#!/default/HGNC?datasets=hgnc_gene_mart) (accession 31.10.2023) \
Locus group: protein-coding gene \
_Approved symbol, UniProt accession_

file_translation_hgnc_ensembl from [ensembl BioMart](https://www.ensembl.org/biomart/martview/834e6d51cb98b100507df185b4d6cadf) (accession 31.10.2023) \
Dataset: GRCh38.p14, Filters: Limit to genes with HGNC Symbol ID(s) \
_Protein stable ID, Gene name_

directory_fasta from David Burke, 10.07.2023

# File paths

In [2]:
# omics data
file_dat_annot = "data/all_samples_annotation.txt"
file_dat_cnv = "data/cnv.txt"
file_dat_tx = "data/transcriptomics_log2fc.txt"
file_dat_px = "data/proteomics.txt"
file_dat_mt = "data/mutations.txt"

# predicted interface data
#file_dat_res = "data/interface_full_strict.txt"
file_dat_res = "data/somatic_mutations_protein_abundance_ifresid.txt"

# database interaction data
file_biogrid_all = "data/biogrid_all.txt"
file_biogrid_mv_physical = "data/biogrid_mv_physical.txt"
file_string_all = "data/string_all.txt"
file_string_physical = "data/string_physical.txt"

# directory with fasta files (for UniProt accessions)
directory_fasta = "data/gene_sequences/"

# gene names / translations
file_translation_hgnc_update = "data/translation_hgnc_update.txt"
file_translation_hgnc_uniprot = "data/translation_hgnc_uniprot.txt"
file_translation_hgnc_ensembl = "data/translation_hgnc_ensembl.txt"

# variant effect predictions
file_dat_vep = "data/cptac_missense_mutations_merge_23.11.1.tsv"

# Utils

In [3]:
# tables for gene name updates / translations
translation_hgnc_update = pd.read_csv(file_translation_hgnc_update, sep="\t")
translation_hgnc_uniprot = pd.read_csv(file_translation_hgnc_uniprot, sep="\t")
translation_hgnc_ensembl = pd.read_csv(file_translation_hgnc_ensembl, sep="\t")

In [4]:
# function to subset omics dataframes based on minimum percentage of non-NA values
def subset_dataframe(df, percent):
    # calculate the percentage of non-NaN values for each gene
    gene_counts = df.count(axis=1)
    gene_percents = gene_counts / df.shape[1] * 100
    # filter based on percent, subset
    valid_genes = gene_percents[gene_percents >= percent].index.tolist()
    df_subset = df.loc[valid_genes]
    
    return df_subset

In [5]:
# function to calculate protein residuals (regressing out RNA) with batch as covariate
def calculate_protein_residuals(df):
    # convert 'batch' into dummy variables
    batch_dummies = pd.get_dummies(df['batch'], drop_first=True).astype(float)
    X = pd.concat([df['value_tx'], batch_dummies], axis=1)
    model = sm.OLS(df['value_px'], sm.add_constant(X))
    result = model.fit()
    return result.resid

In [6]:
# function to calculate correlations between omic types
def calculate_correlation(df, omics1, omics2):
    # Group by 'gene' and calculate correlation for each group
    def group_corr(g):
        return g[omics1].corr(g[omics2])

    correlations = df.groupby('gene').apply(group_corr)
    col_name = f'corr_{omics1}_{omics2}'
    df_return = df.copy()
    df_return[col_name] = df_return['gene'].map(correlations)
    return df_return

In [7]:
def omics_pivot_long(df, omics):
    df.reset_index(inplace=True)
    df_long = pd.melt(df, id_vars='gene')
    df_long.rename(columns={'variable':'sample', 'value':f'value_{omics}'}, inplace=True)
    return df_long

In [8]:
# function to update list of HGNC gene names "approved"
def update_hgnc(gene_list):
    
    # get approved symbols, create dictrionaries
    approved_symbols = set(translation_hgnc_update['Approved symbol'])
    alias_map = pd.Series(translation_hgnc_update['Approved symbol'].values,
                          index=translation_hgnc_update['Alias symbol']).to_dict()
    previous_map = pd.Series(translation_hgnc_update['Approved symbol'].values,
                             index=translation_hgnc_update['Previous symbol']).to_dict()

    # combine dictionaries (if conflict, alias_map will overwrite previous_map), find approved symbol
    gene_map = {**previous_map, **alias_map}
    translated_genes = [(gene, gene_map.get(gene, gene)) for gene in gene_list]
    
    # create DataFrame with old and updated names, set gene_hgnc to 'NA' if not in approved symbols
    translation_table = pd.DataFrame(translated_genes, columns=['gene_hgnc_data', 'gene_hgnc_approved'])
    translation_table['gene_hgnc_approved'] = translation_table['gene_hgnc_approved'].apply(
        lambda x: x if x in approved_symbols else np.nan)

    return translation_table

In [9]:
# function to update HGNC gene names in dataframe to approved and average multi-mappings
def update_hgnc_and_average_values(df):
    # call update_hgnc to update gene names
    gene_list = df.index.tolist()  
    translation_table = update_hgnc(gene_list)
    df_intermediate = df.reset_index().rename(columns={'gene':'gene_hgnc_data'})\
                                        .merge(translation_table, on='gene_hgnc_data')\
                                        .dropna(subset='gene_hgnc_approved').drop('gene_hgnc_data', axis=1)
    df_return = df_intermediate.rename(columns={'gene_hgnc_approved':'gene'}).set_index('gene')
    
    # new mappings cause duplicates -> identify and average
    counts = df_intermediate['gene_hgnc_approved'].value_counts()
    multi_entry_genes = counts[counts > 1].index
    df_multi = df_intermediate[df_intermediate['gene_hgnc_approved'].isin(multi_entry_genes)]
    df_multi_avg = df_multi.groupby('gene_hgnc_approved').mean().reset_index()

    # concatenate the single entry genes with the averaged data for multi-entry genes
    single_entry_genes = counts[counts == 1].index
    df_single = df_intermediate[df_intermediate['gene_hgnc_approved'].isin(single_entry_genes)]
    df_return = pd.concat([df_single, df_multi_avg], 
                        ignore_index=True).rename(columns={'gene_hgnc_approved':'gene'}).set_index('gene')
    return df_return

In [10]:
# Function to translate between STRING ensembl IDs and HGNC
def translate_string(df, translation_table):
    # Remove 9606. prefix
    df["protein1"] = df["protein1"].str.replace("9606.", "", regex=False)
    df["protein2"] = df["protein2"].str.replace("9606.", "", regex=False)
    # Merge on 'protein1'
    df_tl = pd.merge(df, translation_table, left_on='protein1', right_on='Protein stable ID', how='left')
    df_tl.rename(columns={'Gene name': 'interactor_A'}, inplace=True)
    df_tl.drop(['Protein stable ID'], axis=1, inplace=True)

    # Merge on 'protein2'
    df = pd.merge(df_tl, translation_table, left_on='protein2', right_on='Protein stable ID', how='left')
    df.rename(columns={'Gene name': 'interactor_B'}, inplace=True)
    df.drop(['Protein stable ID'], axis=1, inplace=True)
    
    return df[['interactor_A', 'interactor_B']]

In [11]:
# Function based on update_hgnc() adapted for BioGRID
def update_hgnc_biogrid(df, column_name):
    # Create intermediate df with translations
    df_intermediate = update_hgnc(df[column_name].tolist()).drop_duplicates()
    # Merge with updated HGNC names
    df = df.merge(df_intermediate, 
                  left_on=column_name, right_on='gene_hgnc_data', how='left')
    
    # Drop original column and 'gene_hgnc_data'
    df.drop(columns=[column_name, 'gene_hgnc_data'], inplace=True)
    
    # Rename the 'gene_hgnc_approved' column to original column name
    df.rename(columns={'gene_hgnc_approved': column_name}, inplace=True)
    
    return df

In [12]:
# Function to filter & rename biogrid data
def filter_biogrid(df):
    columns_keep = ['Official Symbol Interactor A', 'Official Symbol Interactor B']
    mask = (
        (df['Organism Name Interactor A'] == 'Homo sapiens') &
        (df['Organism Name Interactor B'] == 'Homo sapiens') &
        (df['Experimental System'] != 'Affinity Capture-RNA') &
        (df['Experimental System'] != 'Protein-RNA') &
        (df['Official Symbol Interactor A'] != df['Official Symbol Interactor B'])
    )
    
    return df[mask][columns_keep].drop_duplicates().rename(columns={'Official Symbol Interactor A':'interactor_A',
                                                                    'Official Symbol Interactor B':'interactor_B'})


In [13]:
# Function that makes df with two interactor columns (_A, _B) redundant
def make_redundant(df):
    df_reversed = df.rename(columns={"interactor_A": "interactor_B", "interactor_B": "interactor_A"})
    df_return = pd.concat([df, df_reversed], ignore_index=True).drop_duplicates()
    
    # remove homodimers
    df_return = df_return[df_return['interactor_A']!=df_return['interactor_B']]
    return df_return

In [14]:
# retrieval of full sequence and interface residues from FASTA files (via UniProt accession)
def get_residues_from_fasta(protein_id, residues_list):
    fasta_file = f'{directory_fasta}{protein_id}.fasta'
    
    # check if fasta file exists
    if not os.path.exists(fasta_file):
        return np.nan, np.nan

    with open(fasta_file, 'r') as fasta_handle:
        for record in SeqIO.parse(fasta_handle, 'fasta'):
            protein_sequence = str(record.seq)
            
            # return amino acids if index is within the sequence length, else return np.nan
            try:
                return ''.join([protein_sequence[i-1] for i in residues_list]), protein_sequence
            except IndexError:
                return np.nan, np.nan

In [15]:
# function that fits two nested linear models and evaluates goodness of fit improvement using LRT
# batch is handled as a covariate
def nested_models_lrt(df_omics, df_ixn):
    # preprocess df_omics for faster access
    df_omics_values = df_omics.set_index(['gene', 'sample'])
    
    # find all interactors B of interactor A based on ixn file, save to dictionary
    interactors_dict = defaultdict(list)
    for _, row in df_ixn.iterrows():
        interactors_dict[row['interactor_A']].append(row['interactor_B'])
        
    # utils
    start_time = time.time()    
    results = []
    i = 1
    tot_ixn = len(interactors_dict)
    
    # iterate through all interactors A
    for gene_A, interactors in interactors_dict.items():
        df_omics_A = df_omics_values.loc[gene_A]
        print(f'Progress: {i}/{tot_ixn}', end='\r', flush=True)
        i = i+1
        
        # iterate through all interactors B of interactor A
        for gene_B in interactors:
            df_omics_B = df_omics_values.loc[gene_B]
            merged = df_omics_A.reset_index().merge(df_omics_B.reset_index(), 
                                                    on=['sample', 'batch'], suffixes=('_A', '_B'))
            merged['gene_A'] = gene_A
            merged['gene_B'] = gene_B
            
            # null Model
            X_null = sm.add_constant(
                pd.concat([merged['value_tx_A'], pd.get_dummies(
                    merged['batch'],drop_first=True).astype(float)], axis=1))
            
            if merged['value_px_A'].empty or X_null.empty:
                continue
            model_null = sm.OLS(merged['value_px_A'], X_null)
            results_null = model_null.fit()

            # alternative Model
            X_alt = sm.add_constant(pd.concat([merged[['value_tx_A', 'value_cnv_B']], 
                                               pd.get_dummies(merged['batch'], drop_first=True).astype(float)], axis=1))
            model_alt = sm.OLS(merged['value_px_A'], X_alt)
            results_alt = model_alt.fit()
            
            # likelihood Ratio Test
            LRT = 2 * (results_alt.llf - results_null.llf)
            p_LRT = stats.chi2.sf(LRT, df=1)  # sf is survival function, which is 1 - cdf
            
            # save regression parameters of alternative model and p-values
            params = results_alt.params.values
            merged['param_const'] = params[0] # The first parameter is the intercept (const)
            merged['param_value_tx_A'] = params[1] # The second parameter is for value_tx_A
            merged['param_value_cnv_B'] = params[2] # The third parameter is for value_cnv_B
            merged['p_LRT'] = p_LRT
            
            # drop unused columns, save result
            cols_use = ['gene_A', 'gene_B', 
                        'param_value_cnv_B', 'p_LRT']
            merged = merged[cols_use].drop_duplicates()
            results.append(merged)

    end_time = time.time()
    print(f"\nThe loop took {end_time - start_time} seconds to complete.")

    merged_data = pd.concat(results, ignore_index=True)
    return merged_data

In [16]:
# function to get current date and time
def get_current_datetime_string():
    return datetime.now().strftime("data_processing_%y%m%d_%H%M")

In [17]:
# Function to map three-letter code to one-letter code
def map_to_one_letter(code):
    aa_dict = {
    'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cys': 'C', 'Glu': 'E', 'Gln': 'Q', 'Gly': 'G', 'His': 'H',
    'Ile': 'I', 'Leu': 'L', 'Lys': 'K', 'Met': 'M', 'Phe': 'F', 'Pro': 'P', 'Ser': 'S', 'Thr': 'T', 'Trp': 'W',
    'Tyr': 'Y', 'Val': 'V'
    }
    if isinstance(code, str):
        if code in aa_dict:
            return aa_dict[code]
        elif len(code) == 1:
            return code
    return None

In [18]:
# function to categorize FoldX score
def categorize_foldx(score):
    if pd.isna(score):
        return np.nan
    elif score > foldx_threshold_destabilizing:
        return 'destabilizing'
    elif score < foldx_threshold_stabilizing:
        return 'stabilizing'
    else:
        return 'neutral'

In [19]:
# function to categorize EVE score
def categorize_eve(score):
    if pd.isna(score):
        return np.nan
    elif score > eve_threshold:
        return 'pathogenic'
    else:
        return 'benign'

# Prepare CPTAC data

In [20]:
# read CPTAC data
dat_annot = pd.read_csv(file_dat_annot, sep="\t")
dat_cnv = pd.read_csv(file_dat_cnv, sep="\t")
dat_tx = pd.read_csv(file_dat_tx, sep="\t")
dat_px = pd.read_csv(file_dat_px, sep="\t")

dat_cnv.set_index('gene', inplace=True)
dat_tx.set_index('gene', inplace=True)
dat_px.set_index('gene', inplace=True)

# SUBSET SAMPLES
# define batches to exclude from analysis

#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
exclude_batches = ['ccle', 'cbttc', 'tcga-coread']
#---------------------------------------------------------------------------------------------------------------------------#

samples_tissue = set(dat_annot[~dat_annot['batch'].isin(exclude_batches)]['sample'])

# find common samples, subset
common_samples = list(set(dat_cnv.columns) & set(dat_tx.columns) & 
                      set(dat_px.columns) & samples_tissue)
dat_cnv = dat_cnv[common_samples]
dat_tx = dat_tx[common_samples]
dat_px = dat_px[common_samples]

# SUBSET GENES
# throw away genes with low coverage and low RNA-protein correlation

#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
min_percent = 25
#---------------------------------------------------------------------------------------------------------------------------#

dat_cnv = subset_dataframe(dat_cnv, percent=min_percent)
dat_tx = subset_dataframe(dat_tx, percent=min_percent)
dat_px = subset_dataframe(dat_px, percent=min_percent)

# find common genes, subset
common_genes = list(set(dat_tx.index) & set(dat_px.index) & set(dat_cnv.index))
dat_cnv = dat_cnv.loc[common_genes]
dat_tx = dat_tx.loc[common_genes]
dat_px = dat_px.loc[common_genes]

# UPDATE HGNC SYMBOLS
dat_cnv = update_hgnc_and_average_values(dat_cnv)
dat_tx = update_hgnc_and_average_values(dat_tx)
dat_px = update_hgnc_and_average_values(dat_px)

batch_counts = dat_annot[['sample', 'batch']][dat_annot['sample'].isin(common_samples)].drop_duplicates().batch.value_counts()
df_batch_counts = pd.DataFrame(batch_counts).reset_index()

In [21]:
# pivot omics measurement dfs to long, merge
dat_cnv_long = omics_pivot_long(dat_cnv, omics='cnv')
dat_tx_long = omics_pivot_long(dat_tx, omics='tx')
dat_px_long = omics_pivot_long(dat_px, omics='px')
dat_omics_int = dat_cnv_long.merge(dat_tx_long, on=['gene', 'sample'], how='inner')
dat_omics = dat_omics_int.merge(dat_px_long, on=['gene', 'sample'], how='inner')

# only keep samples with cnv, tx and px measurements
dat_omics.dropna(inplace=True)

# annotate, rearrange
dat_annot = dat_annot[['sample', 'batch']].drop_duplicates()
dat_omics = dat_omics.merge(dat_annot, on='sample', how='left')

order_cols = ['gene', 'sample', 'batch', 'value_cnv', 'value_tx', 'value_px']
dat_omics = dat_omics[order_cols]
dat_omics = dat_omics.sort_values(by=['batch', 'sample', 'gene'])
dat_omics.reset_index(drop=True, inplace=True)

# for each gene, calculate protein residuals
residuals = dat_omics.groupby('gene').apply(calculate_protein_residuals).reset_index(level=0, drop=True)
dat_omics['residuals'] = residuals

# for each gene, calculate correlation between RNA and protein
dat_omics = calculate_correlation(dat_omics, 'value_tx', 'value_px')

# store subsetted & updated gene names for later use
common_genes = dat_omics['gene'].unique().tolist()
common_samples = dat_omics['sample'].unique().tolist()

In [22]:
dat_mt = pd.read_csv(file_dat_mt, sep="\t")

dat_mt.rename(columns={'gene_symbol':'gene'}, inplace=True)

dat_mt = dat_mt[(dat_mt['gene'].isin(common_genes)) & 
                (dat_mt['sample'].isin(common_samples))]

# split the "HGVSp" column into three new columns, standardize to one letter code
dat_mt[['pREF', 'pPOS', 'pALT']] = dat_mt['HGVSp'].str.extract(r'p\.([A-Za-z]+)(\d+)([A-Za-z]+)')
dat_mt['pREF'] = dat_mt['pREF'].str.capitalize().apply(map_to_one_letter)
dat_mt['pALT'] = dat_mt['pALT'].str.capitalize().apply(map_to_one_letter)

dat_mt['pREF'] = dat_mt['pREF'].astype(str)
dat_mt['pPOS'] = dat_mt['pPOS'].astype(str)
dat_mt['pALT'] = dat_mt['pALT'].astype(str)

dat_mt_var_cl = dat_mt[['gene', 'sample', 'variant_class', 'pPOS', 'pREF', 'pALT']].drop_duplicates()

variant_classes = ['Missense_Mutation', 'Frame_Shift_Del', 'Nonsense_Mutation']

dat_mt_var_cl = dat_mt_var_cl[dat_mt_var_cl['variant_class'].isin(variant_classes)]

# If there is a nonsense mutation in a gene, it will have a higher impact 
# on protein abundance/functionality than a co-occurring missense mutation. 
# In these cases, the gene will be regarded as only nonsense-mutated

# Define the priority for each variant class
priority_map = {
    'Nonsense_Mutation': 1,
    'Frame_Shift_Del': 2,
    'Missense_Mutation': 3
}

# Function to get the highest priority variant
def get_highest_priority_variant(group):
    return group.loc[group['variant_class'].map(priority_map).idxmin()]

# Sort by gene, sample, and priority
dat_mt_sort = dat_mt_var_cl.sort_values(by=['gene', 'sample', 'variant_class'], key=lambda x: x.map(priority_map))

# Group by gene and sample, and apply the function to get the highest priority variant
dat_mt_prio = dat_mt_sort.groupby(['gene', 'sample']).apply(get_highest_priority_variant).reset_index(drop=True)
dat_omics_mt_prio = dat_omics.merge(dat_mt_prio, on=['gene', 'sample'], how='inner')

some_mutation = dat_mt[['sample', 'gene']].drop_duplicates()

# Merge with an indicator to find matching rows
dat_omics_no_mt = dat_omics.merge(some_mutation, on=['sample', 'gene'], how='left', indicator=True)

# Filter out rows that are found in both dataframes
dat_omics_no_mt = dat_omics_no_mt[dat_omics_no_mt['_merge'] == 'left_only'].drop(columns=['_merge'])
dat_omics_no_mt['variant_class'] = 'No_Mutation'

dat_omics_mt = pd.concat([dat_omics_mt_prio, dat_omics_no_mt], axis=0)

### Remove hypermutated samples, genes with low RNA-Protein correlation

In [23]:
#Discard genes with low/negative mRNA-protein correlation (can also be further subsetted in data_viz.ipynb)
#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
correlation_cutoff = 0
#---------------------------------------------------------------------------------------------------------------------------#

# subset according to correlation cutoff
dat_omics_mt = dat_omics_mt[dat_omics_mt['corr_value_tx_value_px']>correlation_cutoff]

In [24]:
#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
remove_hypermutated = True
#---------------------------------------------------------------------------------------------------------------------------#

mutations_per_sample = dat_omics_mt[dat_omics_mt['variant_class']!='No_Mutation'].groupby('sample').size().reset_index(name='mutations_count')

# Remove hypermutated samples
hypermutated_samples = mutations_per_sample[mutations_per_sample['mutations_count']>500]['sample']
if remove_hypermutated:
    # remove from mutations dataset
    dat_omics_mt = dat_omics_mt[~dat_omics_mt['sample'].isin(hypermutated_samples)]
    
    # remove from omics dataset
    dat_omics = dat_omics[~dat_omics['sample'].isin(hypermutated_samples)]
    common_samples = dat_omics['sample'].unique().tolist()

# Prepare DB interactions (STRING, BioGRID)

In [25]:
# read STRING data
string_all = pd.read_csv(file_string_all, sep=' ')
string_physical = pd.read_csv(file_string_physical, sep=' ')

# Translate protein IDs to gene names
string_all_translated = translate_string(string_all, translation_hgnc_ensembl)
string_physical_translated = translate_string(string_physical, translation_hgnc_ensembl)

# Drop NAs, duplicates
string_all_translated = string_all_translated.dropna(axis=0).drop_duplicates()
string_physical_translated = string_physical_translated.dropna(axis=0).drop_duplicates()

# Make both redundant
string_all_redundant = make_redundant(string_all_translated)
string_physical_redundant = make_redundant(string_physical_translated)

# Outer merge with set definition
string = pd.merge(string_all_redundant, string_physical_redundant, 
                  on=['interactor_A', 'interactor_B'], how='outer', indicator=True)

string.rename(columns={'_merge':'string_set'}, inplace=True)
string['string_set'] = string['string_set'].map({'both': 'physical', 'left_only': 'general'})

# Filter based on common genes
string = string[(string['interactor_A'].isin(common_genes)) & 
                (string['interactor_B'].isin(common_genes))].drop_duplicates()

In [26]:
# read BioGRID data
biogrid_all = pd.read_csv(file_biogrid_all, sep='\t', low_memory=False)
biogrid_mv_physical = pd.read_csv(file_biogrid_mv_physical, sep='\t', low_memory=False)

# Filter using the new function
biogrid_all = filter_biogrid(biogrid_all)
biogrid_physical = filter_biogrid(biogrid_mv_physical)

# Update interactor_A and interactor_B names for biogrid_all
biogrid_all = update_hgnc_biogrid(biogrid_all, 'interactor_A')
biogrid_all = update_hgnc_biogrid(biogrid_all, 'interactor_B')

# Update interactor_A and interactor_B names for biogrid_physical
biogrid_physical = update_hgnc_biogrid(biogrid_physical, 'interactor_A')
biogrid_physical = update_hgnc_biogrid(biogrid_physical, 'interactor_B')

# Drop NAs, duplicates
biogrid_all = biogrid_all.dropna(axis=0).drop_duplicates()
biogrid_physical = biogrid_physical.dropna(axis=0).drop_duplicates()

# Make both redundant
biogrid_all = make_redundant(biogrid_all)
biogrid_physical = make_redundant(biogrid_physical)

# Outer merge with set definition
biogrid = pd.merge(biogrid_all, biogrid_physical, 
                   on=['interactor_A', 'interactor_B'], how='outer', indicator=True)

biogrid.rename(columns={'_merge':'biogrid_set'}, inplace=True)
biogrid['biogrid_set'] = biogrid['biogrid_set'].map({'both': 'physical', 'left_only': 'general'})

# Filter based on common genes
biogrid = biogrid[(biogrid['interactor_A'].isin(common_genes)) & 
                  (biogrid['interactor_B'].isin(common_genes))].drop_duplicates()

# Prepare interaction data (David)

In [27]:
# read interaction data
dat_res = pd.read_csv(file_dat_res, sep='\t')
dat_res

# only consider moderate-to-high confidence interaction models

#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
min_pdockq = 0.23
#---------------------------------------------------------------------------------------------------------------------------#
dat_res = dat_res[dat_res['pdockq'] > min_pdockq]

# split the dataframe into two: one for chain A and one for chain B
dat_res_A = dat_res[dat_res['chain'] == 'A'].reset_index(drop=True)
dat_res_B = dat_res[dat_res['chain'] == 'B'].reset_index(drop=True)

# rename the columns to distinguish between chain A and chain B
#dat_res_A = dat_res_A.rename(columns={'protein': 'protein1', 'residues': 'residues1'})
#dat_res_B = dat_res_B.rename(columns={'protein': 'protein2', 'residues': 'residues2'})
dat_res_A = dat_res_A.rename(columns={'protein': 'protein1', 'resseq': 'residues1'})
dat_res_B = dat_res_B.rename(columns={'protein': 'protein2', 'resseq': 'residues2'})
dat_res_B = dat_res_B[['protein2', 'residues2']]
dat_res = pd.concat([dat_res_A, dat_res_B], axis=1)

# drop unnecessary columns, rename
#dat_res = dat_res.drop(columns=['chain', 'clashes'])
dat_res = dat_res.drop(columns=['chain'])
dat_res = dat_res[['pair', 'pdockq', 'protein1', 'residues1', 'protein2', 'residues2']]
col_names = ['pair', 'pDockQ', 'protein1', 'residues1', 'protein2', 'residues2']
dat_res.columns = col_names

dat_res.dropna(axis=0, inplace=True)

In [28]:
# apply the function to each row of the data frame for protein1 and protein2
tqdm.pandas(desc="Processing protein1")
dat_res[['residues1_aa', 'sequence1']] = dat_res.progress_apply(
    lambda row: pd.Series(get_residues_from_fasta(row['protein1'], list(map(int, row['residues1'].split(','))))), axis=1)

tqdm.pandas(desc="Processing protein2")
dat_res[['residues2_aa', 'sequence2']] = dat_res.progress_apply(
    lambda row: pd.Series(get_residues_from_fasta(row['protein2'], list(map(int, row['residues2'].split(','))))), axis=1)

dat_res.dropna(axis=0, inplace=True)

# count residues
residues1_lists = dat_res['residues1'].apply(lambda x: list(map(int, x.split(','))))
residues2_lists = dat_res['residues2'].apply(lambda x: list(map(int, x.split(','))))
dat_res['NumRes1'] = residues1_lists.apply(len)
dat_res['NumRes2'] = residues2_lists.apply(len)
dat_res['NumRes'] = dat_res['NumRes1'] + dat_res['NumRes2']

Processing protein2: 100%|███████████████████████████████████████████████████| 103863/103863 [00:48<00:00, 2145.27it/s]


In [29]:
# make dataframe redundant (A->B, B->A)
dat_res_copy = dat_res.copy()

# switch necessary columns in the copied dataframe
dat_res_copy[['protein1', 'protein2']] = dat_res_copy[['protein2', 'protein1']]
dat_res_copy[['residues1', 'residues2']] = dat_res_copy[['residues2', 'residues1']]
dat_res_copy[['residues1_aa', 'residues2_aa']] = dat_res_copy[['residues2_aa', 'residues1_aa']]
dat_res_copy[['NumRes1', 'NumRes2']] = dat_res_copy[['NumRes2', 'NumRes1']]
dat_res_copy[['sequence1', 'sequence2']] = dat_res_copy[['sequence2', 'sequence1']]

# append the copied dataframe to the original dataframe
dat_res = pd.concat([dat_res, dat_res_copy], ignore_index=True)
dat_res.drop_duplicates(inplace=True)

# rename columns
dat_res.rename(columns={'protein1':'interactor_A', 'protein2':'interactor_B', 
                        'residues1':'index_residues_A', 'residues2':'index_residues_B', 
                        'NumRes1':'NumRes_A', 'NumRes2':'NumRes_B', 
                        'residues1_aa':'residues_A', 'residues2_aa':'residues_B', 
                        'sequence1':'sequence_A', 'sequence2':'sequence_B'}, inplace=True)

In [30]:
# prepare translation table
translation_hgnc_uniprot = translation_hgnc_uniprot[['Approved symbol', 'UniProt accession']].drop_duplicates().dropna(axis=0)
# only keep HGNC names that appear in data (avoids surjective mappings)
translation_hgnc_uniprot = translation_hgnc_uniprot[translation_hgnc_uniprot['Approved symbol'].isin(common_genes)]
# throw away injective mappings (very few cases)
translation_hgnc_uniprot = translation_hgnc_uniprot.drop_duplicates(subset='UniProt accession', keep='first')

# translate (tl) interaction data
dat_res_tl = dat_res.merge(translation_hgnc_uniprot, left_on='interactor_A', right_on='UniProt accession', how='left')
dat_res_tl = dat_res_tl.drop(columns=['UniProt accession', 'interactor_A'])
dat_res_tl = dat_res_tl.rename(columns={'Approved symbol': 'interactor_A'})
dat_res_tl = dat_res_tl.merge(translation_hgnc_uniprot, left_on='interactor_B', right_on='UniProt accession', how='left')
dat_res_tl = dat_res_tl.drop(columns=['UniProt accession', 'interactor_B'])
dat_res_tl = dat_res_tl.rename(columns={'Approved symbol': 'interactor_B'})
dat_res_tl.dropna(axis=0, inplace=True)

# drop Homodimers
dat_res_tl = dat_res_tl[dat_res_tl['interactor_A']!=dat_res_tl['interactor_B']]

# translation causes multi-mappings -> average
numeric_cols = ['pDockQ', 'NumRes_A', 'NumRes_B', 'NumRes']
dat_res_numeric = dat_res_tl.groupby(['interactor_A', 'interactor_B'])[numeric_cols].mean().reset_index()
dat_res_numeric['NumRes'] = dat_res_numeric['NumRes'].round().astype(int)
dat_res_numeric['NumRes_A'] = dat_res_numeric['NumRes_A'].round().astype(int)
dat_res_numeric['NumRes_B'] = dat_res_numeric['NumRes_B'].round().astype(int)
dat_res_non_numeric = dat_res_tl.groupby(['interactor_A', 'interactor_B'], as_index=False).first()
dat_res_non_numeric = dat_res_non_numeric.drop(columns=dat_res_numeric.columns[2:])
dat_res_avg = pd.merge(dat_res_numeric, dat_res_non_numeric, on=['interactor_A', 'interactor_B'])

# order, drop columns
cols_use = ['interactor_A', 'interactor_B', 'NumRes_A', 'NumRes_B', 'NumRes', 'pDockQ', 
            'residues_A', 'residues_B', 'index_residues_A', 'index_residues_B', 'sequence_A', 'sequence_B']

dat_ixn = dat_res_avg[cols_use]
dat_ixn.reset_index(drop=True, inplace=True)

In [31]:
# compile database info
db_info = string.merge(biogrid, on=['interactor_A', 'interactor_B'], how='outer')

# merge with database information
dat_ixn = dat_ixn.merge(string, on=['interactor_A', 'interactor_B'], how='left')
dat_ixn['string_set'] = dat_ixn['string_set'].fillna('None')

dat_ixn = dat_ixn.merge(biogrid, on=['interactor_A', 'interactor_B'], how='left')
dat_ixn['biogrid_set'] = dat_ixn['biogrid_set'].fillna('None')

In [32]:
db_physical = db_info[(db_info['string_set']=='physical') | (db_info['biogrid_set']=='physical')]
no_db_physical = db_physical.shape[0]/2

In [33]:
# only consider interactions with additional database evidence

#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
string_sets = ['physical']
biogrid_sets = ['physical']
#---------------------------------------------------------------------------------------------------------------------------#

# Define logic as lambda functions
AND_logic = lambda x, y: x & y
OR_logic = lambda x, y: x | y

# Determine which logic to use

#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
logic = OR_logic
#---------------------------------------------------------------------------------------------------------------------------#

# Save logic as string for parameter export
if logic == AND_logic:
    logic_str = "AND"
else:
    logic_str = "OR"

# subset dat_ixn
dat_ixn = dat_ixn[logic(dat_ixn['string_set'].isin(string_sets), dat_ixn['biogrid_set'].isin(biogrid_sets))]

# find no of unique interactions
unique_ixn = dat_ixn.shape[0]/2

# Nested linear models, LRT

In [34]:
# INCREASE MEMORY EFFICIENCY
# convert float64 to float32
float64_cols = dat_omics.select_dtypes(include='float64').columns
dat_omics[float64_cols] = dat_omics[float64_cols].astype('float32')

# convert batch to category
dat_omics['batch'] = dat_omics['batch'].astype('category')

# drop unused columns
columns_use = ['gene', 'sample', 'batch', 'value_cnv', 'value_tx', 'value_px', 'residuals', 'corr_value_tx_value_px']
dat_omics = dat_omics[columns_use]

In [35]:
dat_lrt = nested_models_lrt(df_omics=dat_omics, df_ixn=dat_ixn)

Progress: 7170/7170
The loop took 1151.940081834793 seconds to complete.


In [36]:
dat_lrt = dat_lrt.merge(dat_ixn, 
                        left_on=['gene_A', 'gene_B'], 
                        right_on=['interactor_A', 'interactor_B'], 
                        how='inner').drop(['interactor_A', 'interactor_B'], axis=1)

# adjust p-values for false discovery rate using the Benjamini-Hochberg method
dat_lrt['p_LRT_adj'] = multipletests(dat_lrt['p_LRT'].dropna().values, method='fdr_bh')[1]
dat_lrt['neg_log10_p_LRT_adj'] = -np.log10(dat_lrt['p_LRT_adj'])

# Stability

In [37]:
# thresholds for classifying into deleterious/non-deleterious
#-------------------------------------------------------- PARAMETER --------------------------------------------------------#
foldx_threshold_destabilizing = 2
foldx_threshold_stabilizing = -2
eve_threshold = 0.7
#---------------------------------------------------------------------------------------------------------------------------#

In [38]:
# read VEP file
dat_vep = pd.read_csv(file_dat_vep, sep="\t")

# subset & rename columns
keep_columns = ['sample', 'gene', 'pPOS', 'pREF', 'pALT', 
                'ESM1b_LLR', 'ESM1b_is_pathogenic', 'am_pathogenicity', 'am_class', 'eve', 'pred_ddg', 'plddt']
dat_vep = dat_vep[keep_columns]

dat_vep.rename(columns={'ESM1b_LLR':'ESM_score', 'ESM1b_is_pathogenic':'ESM_cat', 
                        'am_pathogenicity':'AM_score', 'am_class':'AM_cat', 
                        'eve':'EVE_score', 'pred_ddg':'FoldX_score', 'plddt':'pLDDT'}, inplace=True)

# subset common samples and common genes
dat_vep = dat_vep[(dat_vep['gene'].isin(common_genes)) & (dat_vep['sample'].isin(common_samples))]

# -99 is NA
dat_vep['EVE_score'] = dat_vep['EVE_score'].replace(-99, np.nan)
dat_vep['pLDDT'] = dat_vep['pLDDT'].replace(-99, np.nan)
dat_vep['FoldX_score'] = dat_vep['FoldX_score'].replace(-99, np.nan)

# drop empty data
dat_vep_null = dat_vep[dat_vep[['ESM_score', 'ESM_cat', 
                                'AM_score', 'AM_cat', 
                                'EVE_score', 'FoldX_score', 'pLDDT']].isnull().all(axis=1)]
dat_vep = dat_vep.drop(dat_vep_null.index)

# Apply categorize_foldx to create a new column 'FoldX_cat'
dat_vep['FoldX_cat'] = dat_vep['FoldX_score'].apply(categorize_foldx)
dat_vep['EVE_cat'] = dat_vep['EVE_score'].apply(categorize_eve)

# In some samples, on gene carries multiple mutations at different position -> only keep most deleterious
multiple_mutations = dat_vep[dat_vep.duplicated(subset=['gene', 'sample'], keep=False)]
multiple_mutations_sorted = multiple_mutations.sort_values(by=['gene', 'sample', 'AM_score'], 
                                                           ascending=[True, True, False])
multiple_mutations_filtered = multiple_mutations_sorted.drop_duplicates(subset=['gene', 'sample'], keep='first')
dat_vep = dat_vep.drop(multiple_mutations.index)
dat_vep = pd.concat([dat_vep, multiple_mutations_filtered], ignore_index=True)

# merge vep and omics data
dat_vep_omics = pd.merge(dat_vep, dat_omics, on=['gene', 'sample'], how='left')
# drop vep data without omics measurements
dat_vep_omics_null = dat_vep_omics[dat_vep_omics[['value_cnv', 'value_tx', 'value_px']].isnull().all(axis=1)]
dat_vep_omics = dat_vep_omics.drop(dat_vep_omics_null.index)

dat_vep_omics = dat_vep_omics[dat_vep_omics['corr_value_tx_value_px']>correlation_cutoff]

# Data Export

In [39]:
# create folder name and path using current date and time
folder_name = f"{get_current_datetime_string()}"
full_path = os.path.join("data_computed", folder_name)
os.makedirs(full_path, exist_ok=True)

# save the files
dat_omics.to_csv(os.path.join(full_path, "omics.txt"), sep="\t", index=False)
dat_omics_mt.to_csv(os.path.join(full_path, "omics_mt.txt"), sep="\t", index=False)
dat_lrt.to_csv(os.path.join(full_path, "lrt.txt"), sep="\t", index=False)
dat_vep_omics.to_csv(os.path.join(full_path, "vep_omics.txt"), sep="\t", index=False)
df_batch_counts.to_csv(os.path.join(full_path, "batch_counts.txt"), sep="\t", index=False)

# write parameters used for this analysis to file
params_path = os.path.join(full_path, "parameters.txt")

# Write the parameters to parameters.txt
with open(params_path, 'w') as f:
    f.write(f"min_pdockq: {min_pdockq}\n")
    f.write('\n')
    f.write(f"Interaction in STRING ({', '.join(string_sets)}) {logic_str} in BioGRID ({', '.join(biogrid_sets)})\n")
    f.write('\n')
    f.write(f'No. unique interactions analyzed: {unique_ixn}')
    f.write('\n')
    f.write(f'Correlation cutoff: {correlation_cutoff}')
    f.write('\n')
    if remove_hypermutated:
        f.write('Remove hypermutated samples: True')
    else: 
        f.write('Remove hypermutated samples: False')